# 1. Giới thiệu bài toán
Bài toán: Nhận diện gương mặt của các sinh viên trong lớp học. Với khoảng 23 nhân vật, mỗi người có 2 - 3 ảnh.
Input: Ảnh gương mặt của 1 người.
Ouput: Label của gương mặt đó.
Dataset: 
    - 23 người. Với mỗi người có 1-3 guơng mặt.
    - Các faces đã được cropped, do đó Face Detection (MTCNN) bỏ qua.
    - Do data khá ít, không đủ để  đánh giá. Do đó trong bài toán này, ta thực hiện thêm bước Data Augmentation nhằm làm tăng sự đa dạng của các ảnh -> Sử dụng lib python (imgaug) thực hiện flipping, cropping, transformation, add noise, Gaussian Blur ...
    - 70% train, 30% test.
    - Total images of dataset: 280 images.
Library & language:
    - Python
    - Keras
    - OpenCV
    - Scikit Learn


# 2. Ý tưởng và phương pháp
Từ tập dataset, ta sẽ rút trich các feature của gương mặt bằng việc sử dụng deep feature (VGG16, VGG19).

<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16.png">

<img src="https://cdn-images-1.medium.com/max/1600/1*cufAO77aeSWdShs3ba5ndg.jpeg">

Deep feature được rút trích FC6 (lớp kế cuối) dimension 4096, ta sẽ dùng các method classifiers như SVM, KNN.

SVM: Tìm đường để classify các class
<img src="https://cdn-images-1.medium.com/max/1600/1*nUpw5agP-Vefm4Uinteq-A.png">

K nearest neighbor: sẽ chọn ra k data point để quyết định xem data đó thuộc vào label nào.
<img src="http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/KNN_final1_ibdm8a.png">

Ngoài ra ta cần bổ sung thêm data (Data Augmentation) bằng các phép biến đổi (Affine, Gaussian Blur, Add Noise, Transoformation, Flipping ...)

<img src="https://raw.githubusercontent.com/aleju/imgaug-doc/master/readme_images/examples_grid.jpg">

Sau khi prepare dataset, ta sẽ rút trích deep feature và đưa vào các model classifier để predict face thuộc label nào. Bước cuối cùng ta sẽ đánh giá kết quả. Vì data khá ít, nên trong bài toán này ta sẽ sử dụng SVM và KNN để  classify.

<img src="https://cdn-images-1.medium.com/max/1200/1*mIEg0YdM_lt4Gp4455kaQQ.png">




# 3. Main Source Code


Define các library và model cần thiết

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import svm
from confusion_matrix import plot_confusion_matrix, call_plt
import pickle
import cv2
import numpy as np
import os

modelVGG16 = VGG16(weights='imagenet', include_top=True)
modelVGG16 = Model(input=modelVGG16.input, output=modelVGG16.get_layer('fc2').output)

modelVGG19 = VGG19(weights='imagenet', include_top=True)
modelVGG19 = Model(input=modelVGG19.input, output=modelVGG19.get_layer('fc2').output)

from imgaug import augmenters as iaa



Function support for data augmentation.

In [ ]:
def image_augmentor(img, name_img):
    seqs = [
        iaa.Sequential([
            iaa.SaltAndPepper(p=0.03),
        ]),
        iaa.Sequential([
            iaa.Invert(0.1, per_channel=True),
            iaa.GaussianBlur(sigma=(0,2.0))
        ]),
        iaa.Sequential([
            iaa.Fliplr(0.5), 
            iaa.Invert(0.05, per_channel=True),
            iaa.GaussianBlur(sigma=(0,3.0)) 
        ]),
        iaa.Sequential([
            iaa.MotionBlur(k=5)
        ])
    ]
    step = 0
    for seq in seqs:
        images_aug = seq.augment_images(img)            
        imgPath = name_img + '_aug_' + str(step) + '.jpg'
        cv2.imwrite(imgPath, images_aug)
        step += 1

def data_augmentation(dataset):
    listName = os.listdir(dataset)
    for name in listName:
        imgList = os.listdir(os.path.join(dataset, name))
        print ('This is name: ', name)
        for imgName in imgList:
            path = os.path.join(name, imgName)
            imgPath = os.path.join(dataset, path)
            img = cv2.imread(imgPath)
            image_augmentor(img, imgPath)
            print (imgPath)

Extract feature with VGG16 and VGG19, utils function.

In [ ]:
def extractVGG16(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return modelVGG16.predict(x).reshape((4096,1))

def extractVGG19(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return modelVGG19.predict(x).reshape((4096,1))

def saveDict(obj, pathFile):
    pickle_out = open(pathFile, "wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def loadDict(pathFile):
    pickle_in = open(pathFile, "rb")
    return pickle.load(pickle_in)

Prepare dataset (extract feature VGG16, VGG19) and save into dataset_vgg16.pkl, and dataset_vgg19.pkl.
Training and evaluation with two major classifiers are SVM and KNN.

In [ ]:
def load_img(dataset):
    listName = os.listdir(dataset)
    list_features = []
    list_labels = []
    for name in listName:
        if name.endswith('.pkl'):
            continue
        imgList = os.listdir(os.path.join(dataset, name))
        print ('This is name: ', name)
        for imgName in imgList:
            path = os.path.join(name, imgName)
            imgPath = os.path.join(dataset, path)
            img = image.load_img(imgPath, target_size=(224, 224))
            list_labels.append(name)
            list_features.append(extractVGG19(img))
            print ('This is imgPath: ', imgPath)

    json = {
        'list_features': list_features,
        'list_labels': list_labels
    }
    saveDict(json, 'dataset.pkl')

def train_model(dataset_feature, method, title):
    json = loadDict(dataset_feature)

    X = [item.reshape(-1) for item in json['list_features']]
    y = json['list_labels']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

    if method == 'knn':
        clf = KNeighborsClassifier(n_neighbors=3)
        clf.fit(X_train, y_train)
    elif method == 'svm':
        clf = SVC(kernel='linear')
        clf.fit(X_train, y_train) 

    predict_list = clf.predict(X_test)
    target_names = set(y_test)

    plot_confusion_matrix(y_test, predict_list, classes=target_names, title=title, normalize=True)    
    print classification_report(y_test, predict_list, target_names=target_names)

Main function for running

In [ ]:
if __name__ == '__main__':
    # load_img('train')
    train_model('dataset_vgg16.pkl', 'knn', 'VGG16-KNN')
    train_model('dataset_vgg19.pkl', 'knn', 'VGG19-KNN')
    train_model('dataset_vgg16.pkl', 'svm', 'VGG16-SVM')
    train_model('dataset_vgg19.pkl', 'svm', 'VGG19-SVM')

    # data_augmentation('train')
    # image_augmentor(cv2.imread('elephant.jpg'))


Output Result:

In [ ]:
/home/mrwen/anaconda2/envs/vision/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
2019-04-20 23:04:20.833162: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
main.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fc..., inputs=Tensor("in...)`
  modelVGG16 = Model(input=modelVGG16.input, output=modelVGG16.get_layer('fc2').output)
main.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fc..., inputs=Tensor("in...)`
  modelVGG19 = Model(input=modelVGG19.input, output=modelVGG19.get_layer('fc2').output)
Normalized confusion matrix
('TITLE: ', 'VGG16-KNN')
/home/mrwen/anaconda2/envs/vision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
             precision    recall  f1-score   support

     TAIHPT       0.25      1.00      0.40         3
     NHANTH       0.11      0.50      0.17         4
      TANPV       0.60      0.50      0.55         6
      DUYNN       0.20      0.25      0.22         4
       MYNH       0.00      0.00      0.00         2
      HANTQ       0.00      0.00      0.00         1
       TULG       1.00      0.57      0.73         7
       HOBV       0.00      0.00      0.00         4
     MINHHA       0.67      0.40      0.50         5
      ANHLT       0.00      0.00      0.00         6
       VULQ       0.50      1.00      0.67         1
    THAODNT       1.00      0.33      0.50         3
    DUYETLV       1.00      0.17      0.29         6
     QUANVM       0.00      0.00      0.00         1
    CUONGNT       1.00      0.25      0.40         4
     HOAISD       1.00      0.33      0.50        12
       VUND       0.00      0.00      0.00         3
      LOCTH       0.40      0.57      0.47         7
      TANTD       0.20      0.33      0.25         3
     HUNGNV       0.00      0.00      0.00         2

avg / total       0.53      0.33      0.35        84

Normalized confusion matrix
('TITLE: ', 'VGG19-KNN')
             precision    recall  f1-score   support

     TAIHPT       0.33      0.67      0.44         3
     NHANTH       0.00      0.00      0.00         4
      TANPV       1.00      0.67      0.80         6
      DUYNN       0.25      0.50      0.33         4
       MYNH       0.00      0.00      0.00         2
      HANTQ       0.00      0.00      0.00         1
       TULG       0.80      0.57      0.67         7
       HOBV       1.00      0.25      0.40         4
     MINHHA       0.50      0.20      0.29         5
      ANHLT       0.12      0.17      0.14         6
       VULQ       0.25      1.00      0.40         1
    THAODNT       1.00      0.33      0.50         3
    DUYETLV       0.50      0.17      0.25         6
     QUANVM       0.00      0.00      0.00         1
    CUONGNT       0.00      0.00      0.00         4
     HOAISD       1.00      0.33      0.50        12
       VUND       0.00      0.00      0.00         3
      LOCTH       0.50      0.57      0.53         7
      TANTD       0.20      0.33      0.25         3
     HUNGNV       1.00      0.50      0.67         2

avg / total       0.54      0.33      0.37        84

Normalized confusion matrix
('TITLE: ', 'VGG16-SVM')
             precision    recall  f1-score   support

     TAIHPT       0.50      1.00      0.67         3
     NHANTH       0.38      0.75      0.50         4
      TANPV       0.71      0.83      0.77         6
      DUYNN       1.00      0.75      0.86         4
       MYNH       0.00      0.00      0.00         2
      HANTQ       0.00      0.00      0.00         1
       TULG       1.00      1.00      1.00         7
       HOBV       0.50      0.50      0.50         4
     MINHHA       1.00      0.60      0.75         5
      ANHLT       0.56      0.83      0.67         6
       VULQ       1.00      1.00      1.00         1
    THAODNT       0.00      0.00      0.00         3
    DUYETLV       1.00      0.83      0.91         6
     QUANVM       0.50      1.00      0.67         1
    CUONGNT       1.00      0.25      0.40         4
     HOAISD       1.00      0.67      0.80        12
       VUND       0.00      0.00      0.00         3
      LOCTH       0.70      1.00      0.82         7
      TANTD       0.22      0.67      0.33         3
     HUNGNV       1.00      0.50      0.67         2

avg / total       0.71      0.68      0.66        84

/home/mrwen/Desktop/jupyter/master_cv/face-recognition-mini-challenge/confusion_matrix.py:28: RuntimeWarning: invalid value encountered in divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
Normalized confusion matrix
/home/mrwen/anaconda2/envs/vision/lib/python2.7/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
('TITLE: ', 'VGG19-SVM')
/home/mrwen/anaconda2/envs/vision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 21, does not match size of target_names, 20
  .format(len(labels), len(target_names))
/home/mrwen/anaconda2/envs/vision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
             precision    recall  f1-score   support

     TAIHPT       1.00      0.67      0.80         3
     NHANTH       0.43      0.75      0.55         4
      TANPV       0.86      1.00      0.92         6
      DUYNN       0.50      0.50      0.50         4
       MYNH       0.00      0.00      0.00         2
      HANTQ       0.00      0.00      0.00         1
       TULG       1.00      1.00      1.00         7
       HOBV       0.75      0.75      0.75         4
     MINHHA       0.75      0.60      0.67         5
      ANHLT       0.62      0.83      0.71         6
       VULQ       1.00      1.00      1.00         1
    THAODNT       1.00      0.33      0.50         3
    DUYETLV       1.00      0.67      0.80         6
     QUANVM       1.00      1.00      1.00         1
    CUONGNT       1.00      0.50      0.67         4
     HOAISD       1.00      0.67      0.80        12
       VUND       0.00      0.00      0.00         3
      LOCTH       0.00      0.00      0.00         0
      TANTD       0.70      1.00      0.82         7
     HUNGNV       0.11      0.33      0.17         3

avg / total       0.76      0.68      0.69        84


# 4. Nhận Xét & Đánh Giá

# Confusion Matrix with method KNN (VGG16, VGG19)

<table>
<tr>
  <td> <img src="VGG16-KNN.png" alt="Drawing" style="width: 100%;"/> </td>
  <td> <img src="VGG19-KNN.png" alt="Drawing" style="width: 100%;"/> </td>
</tr>
</table>


# Confusion Matrix with method SVM (VGG16, VGG19)
<table>
<tr>
  <td> <img src="VGG16-SVM.png" alt="Drawing" style="width: 100%;"/> </td>
  <td> <img src="VGG19-SVM.png" alt="Drawing" style="width: 100%;"/> </td>
</tr>
</table>


# 5. Tổng kết